In [1]:
import pandas as pd
import os
import sys

# suppress pandas purely educational warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import pull_csv_from_directory, upload_csv_aws
from scripts.utils.write_metadata import append_metadata

In [2]:
# pull csv from aws
bucket_name = 'ca-climate-index'
aws_dir = '1_pull_data/society_economy/vulnerable_populations/climate_change_health_vulnerabilities/'

pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

Saved DataFrame as 'selectedCHVIdata.csv'


In [3]:
# read in food access data (already subsetted for CA)
cchvi_data = pd.read_csv('selectedCHVIdata.csv')
print(len(cchvi_data))
#os.remove('selectedCHVIdata.csv')

1128364


In [4]:
cchvi_data

,County,FIPS,Region,Definition,Strata,Race,Year,Mean,LL95,UL95,Numerator,Denominator
0,Alameda,6001400100,Bay Area,Annual Mean Ambient Concentration of Fine Part...,none,White,2012-2014,8.697944,NaN,NaN,NaN,2086.0
1,Alameda,6001400100,Bay Area,Annual Mean Ambient Concentration of Fine Part...,none,Total,2012-2014,8.697944,NaN,NaN,NaN,2952.0
2,Alameda,6001400100,Bay Area,Annual Mean Ambient Concentration of Fine Part...,none,AIAN,2009-2011,7.797807,NaN,NaN,NaN,2.0
3,Alameda,6001400100,Bay Area,Annual Mean Ambient Concentration of Fine Part...,none,Asian,2009-2011,7.797807,NaN,NaN,NaN,456.0
4,Alameda,6001400100,Bay Area,Annual Mean Ambient Concentration of Fine Part...,none,AfricanAm,2009-2011,7.797807,NaN,NaN,NaN,140.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1128359,Yuba,6115041100,Northern Central Valley,Population living in sea level rise inundation...,none,Total,2009,0.000000,NaN,NaN,0.0,4941.0
1128360,Yuba,6115041100,Northern Central Valley,Poverty Rate (200% FPL),Overall,White,2011-2015,39.876265,30.344324,49.408207,1418.0,3556.0
1128361,Yuba,6115041100,Northern Central Valley,Poverty Rate (200% FPL),Overall,Total,2011-2015,43.177047,30.793378,55.560716,1851.0,4287.0
1128362,Yuba,6115041100,Northern Central Valley,Projected number of extreme heat days 2040-2060,2040-2060,Total,2040-2060,30.100000,NaN,NaN,36.2,NaN


In [9]:
cchvi_data = cchvi_data[cchvi_data['Race']== 'Total']
unique_entries = cchvi_data['Definition'].unique()
unique_entries

array(['Annual Mean Ambient Concentration of Fine Particulate Matter (PM2.5)',
       'Average Daily Maximum Ozone Concentration',
       'Number of Violent Crimes per 1,000 Population',
       'Percent impervious surface cover',
       'Percent of adults with less than college education',
       'Percent of households with no one aged > 14 years speaking English',
       'Percent of households with no vehicle ownership',
       'Percent of households without air conditioning',
       'Percent of population age less than 5 years',
       'Percent of population aged 65 years or older',
       'Percent of population currently living in very high wildfire risk areas',
       'Percent of population employed and aged > 16 working outdoors',
       'Percent of population with a disability',
       'Percent of population without health insurance',
       'Percent without tree canopy coverage',
       'Population living in sea level rise inundation areas',
       'Poverty Rate (200% FPL)',
   

In [14]:
unique_entries = cchvi_data['Year'].unique()
unique_entries

array(['2012-2014', '2009-2011', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013', '2016', '2006-2010', '2011-2015', '2008-2012', '2009-2013',
       '2040-2060', '2080-2099', nan], dtype=object)

In [42]:
selected_entries = ['2040-2060',
                    '2080-2099',
                    ]

cchvi_data_filtered = cchvi_data[~cchvi_data['Year'].isin(selected_entries)]
# Drop rows with NaN values in 'Year' column
cchvi_data_filtered = cchvi_data_filtered.dropna(subset=['Year'])
unique_entries = cchvi_data_filtered['Year'].unique()

unique_entries

array(['2012-2014', '2009-2011', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013', '2016', '2006-2010', '2011-2015', '2008-2012', '2009-2013'],
      dtype=object)

In [43]:
selected_entries = ['Number of Violent Crimes per 1,000 Population', 
                    'Percent of households without air conditioning', 
                    'Percent of population employed and aged > 16 working outdoors']

cchvi_data_filtered = cchvi_data_filtered[cchvi_data_filtered['Definition'].isin(selected_entries)]
cchvi_data_filtered

,County,FIPS,Region,Definition,Strata,Race,Year,Mean,LL95,UL95,Numerator,Denominator
13,Alameda,6001400100,Bay Area,"Number of Violent Crimes per 1,000 Population",ViolentCrime,Total,2000,6.582206,6.449864,6.714547,9503.000000,1.443741e+06
14,Alameda,6001400100,Bay Area,"Number of Violent Crimes per 1,000 Population",ViolentCrime,Total,2001,6.407031,6.277658,6.536403,9422.000000,1.470572e+06
15,Alameda,6001400100,Bay Area,"Number of Violent Crimes per 1,000 Population",ViolentCrime,Total,2002,6.457854,6.329112,6.586596,9666.000000,1.496782e+06
16,Alameda,6001400100,Bay Area,"Number of Violent Crimes per 1,000 Population",ViolentCrime,Total,2003,6.791432,6.659006,6.923857,10104.000000,1.487757e+06
17,Alameda,6001400100,Bay Area,"Number of Violent Crimes per 1,000 Population",ViolentCrime,Total,2004,6.166925,6.040314,6.293536,9114.000000,1.477884e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
1128272,Yuba,6115041100,Northern Central Valley,"Number of Violent Crimes per 1,000 Population",ViolentCrime,Total,2013,3.715238,3.273710,4.156766,272.000000,7.321200e+04
1128304,Yuba,6115041100,Northern Central Valley,Percent of households without air conditioning,none,Total,2009,1.725293,0.000000,3.798216,756.431724,4.384366e+04
1128305,Yuba,6115041100,Northern Central Valley,Percent of households without air conditioning,none,Total,2009,1.725293,0.000000,3.798216,756.431724,4.384366e+04
1128330,Yuba,6115041100,Northern Central Valley,Percent of population employed and aged > 16 w...,none,Total,2011-2015,8.034611,NaN,NaN,130.000000,1.618000e+03


In [44]:
grouping_cchvi = cchvi_data_filtered.groupby(['FIPS', 'Definition'])['Mean'].mean().reset_index()
grouping_cchvi.head(50)

,FIPS,Definition,Mean
0,6001400100,"Number of Violent Crimes per 1,000 Population",7.267424
1,6001400100,Percent of households without air conditioning,64.306645
2,6001400100,Percent of population employed and aged > 16 w...,0.919842
3,6001400200,"Number of Violent Crimes per 1,000 Population",7.267424
4,6001400200,Percent of households without air conditioning,64.306645
5,6001400200,Percent of population employed and aged > 16 w...,1.595745
6,6001400300,"Number of Violent Crimes per 1,000 Population",7.267424
7,6001400300,Percent of households without air conditioning,64.306645
8,6001400300,Percent of population employed and aged > 16 w...,1.152702
9,6001400400,"Number of Violent Crimes per 1,000 Population",7.267424


In [48]:
pivot_table = grouping_cchvi.pivot_table(index='FIPS', columns='Definition', values='Mean', aggfunc='mean')
pivot_table.reset_index(inplace=True)
random_rows = pivot_table.sample(n=50)
random_rows

Definition,FIPS,"Number of Violent Crimes per 1,000 Population",Percent of households without air conditioning,Percent of population employed and aged > 16 working outdoors
1245,6037111204,6.666117,33.890887,2.969533
2388,6037407801,6.666117,33.890887,7.160561
3041,6037573401,6.666117,33.890887,8.285573
20,6001402500,7.267424,64.306645,2.729833
44,6001404600,7.267424,64.306645,1.086610
3504,6037980026,6.666117,33.890887,0.000000
2984,6037570002,6.666117,33.890887,4.968556
6820,6083000200,3.840452,77.230916,3.420847
5462,6071004601,4.971106,9.455483,4.734935
3252,6037670003,6.666117,33.890887,5.500416


In [56]:
county_tract = "s3://ca-climate-index/0_map_data/ca_tracts_county.csv"
ca_county_tract = pd.read_csv(county_tract)
ca_county_tract = ca_county_tract.rename(columns={'TRACT': 'FIPS'})
ca_county_tract

,Unnamed: 0,FIPS,COUNTYFP,County
0,0,6085504321,85,Santa Clara
1,1,6085504410,85,Santa Clara
2,2,6085507003,85,Santa Clara
3,3,6085507004,85,Santa Clara
4,4,6085502204,85,Santa Clara
...,...,...,...,...
9124,9124,6059001303,59,Orange
9125,9125,6059001304,59,Orange
9126,9126,6059001401,59,Orange
9127,9127,6013367200,13,Contra Costa


In [60]:
cchvi_ca_counties = pd.merge(pivot_table, ca_county_tract[['FIPS', 'County']], on='FIPS', how='left')
column_to_move = 'County'
col = cchvi_ca_counties.pop(column_to_move)
cchvi_ca_counties.insert(1, column_to_move, col)

cchvi_ca_counties.head(40)

,FIPS,County,"Number of Violent Crimes per 1,000 Population",Percent of households without air conditioning,Percent of population employed and aged > 16 working outdoors
0,6001400100,Alameda,7.267424,64.306645,0.919842
1,6001400200,Alameda,7.267424,64.306645,1.595745
2,6001400300,Alameda,7.267424,64.306645,1.152702
3,6001400400,Alameda,7.267424,64.306645,2.146272
4,6001400500,Alameda,7.267424,64.306645,3.771252
5,6001400600,Alameda,7.267424,64.306645,1.753964
6,6001400700,Alameda,7.267424,64.306645,4.000345
7,6001400800,Alameda,7.267424,64.306645,3.738267
8,6001400900,Alameda,7.267424,64.306645,4.275759
9,6001401000,Alameda,7.267424,64.306645,2.052350


In [ ]:
selected_entries = ['Number of Violent Crimes per 1,000 Population', 
                    'Percent of households without air conditioning', 
                    'Percent of population employed and aged > 16 working outdoors']